## Text to image generation using Stable Diffusion

### Installing the dependencies

In [12]:
!pip install diffusers==0.3.0 --q
!pip install transformers scipy ftfy --q
!pip install kaggle --q
!pip install "ipywidgets>=7,<8" --q
import IPython.display 

### Importing the libraries

In [37]:
import gc
import torch
from PIL import Image
import IPython.display 
from torch import autocast
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers import LMSDiscreteScheduler , PNDMScheduler
import os


# Set the Hugging Face API token in the environment
set_env(api_token=api_token)

# Now you can use Hugging Face APIs without importing kaggle_secrets
# Example usage
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")



from transformers.hf_api import get_token

# Set your Hugging Face API token
api_token = "your_api_token_here"

# Set the API token for Hugging Face
os.environ["HF_HOME"] = os.path.join(os.getcwd(), ".huggingface")
get_token(api_token=api_token)

NameError: name 'set_env' is not defined

### Hyperparameter Tuning

In [15]:
class config : 
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    HEIGHT = 512                        
    WIDTH = 512                         
    NUM_INFERENCE_STEPS = 500            
    GUIDANCE_SCALE = 7.5                
    GENERATOR = torch.manual_seed(48)   
    BATCH_SIZE = 1

### Helper Function

In [16]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    w,h = img[0].size
    grid = Image.new('RGB', size = (cols*w, rows*h))
    grid_w, grid_h = grid.size
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

### Pipeline -- loading the pretrained models

In [21]:
from transformers import pipeline

vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", use_auth_token=Hugging_face)
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", use_auth_token=Hugging_face)
vae = vae.to(config.DEVICE)
text_encoder = text_encoder.to(config.DEVICE)
unet = unet.to(config.DEVICE) 

NameError: name 'Hugging_face' is not defined